# Demonstration of Continuous data event recognition using Semantic Segmentation Models

## Import Dependencies

In [14]:
import sys
import zipfile
import os
import numpy as np
import matplotlib.pyplot as plt
from torch import tensor, cuda
import pandas as pd
import requests


### Add utils to sys to import our custom scripts
**Note:** This notebook should be run from the `examples` folder to ensure the correct imports and file paths are used.

In [15]:
# Go up one level from 'examples' and point to 'utils' folder
utils_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'utils'))
# Add the 'utils' folder to sys.path
sys.path.append(utils_path)
models_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'models'))
# Add the 'models' folder to sys.path
sys.path.append(models_path)
import data_utils
import model_utils

### Load the model

Model can be loaded through the model_selector() function. Available architectures are: "UNet", "UNetPlusPlus", "DeepLabV3", "SwinUNet", and PhaseNet. If you wish to load the weights generated by the article, you can set the pretrained argument to True, it automatically downloads the weights from [![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.15098817.svg)](https://doi.org/10.5281/zenodo.15098817) (weights are only available for input size N=256). Number of parameters for each model at each input size evaluated in the article is shown below:

In [16]:
device = "cuda" if cuda.is_available() == True else "cpu"
model = model_utils.model_selector(arch='UNet', N=256, pretrained=True).to(device)

UNet.pt Already downloaded
Loading weights...
Succesfuly loaded weights


### Load continuous trace

Data can be downloaded from [![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.15384890.svg)](https://doi.org/10.5281/zenodo.15384890):

In [17]:
doi = "10.5281/zenodo.15384890"
record_id = doi.split(".")[-1]
metadata_url = f"https://zenodo.org/api/records/{record_id}"
response = requests.get(metadata_url)
metadata = response.json()
files = metadata["files"]


In [18]:
file_to_download = files[0]  # 0: NVCh_10h_continuous_trace.zip - https://zenodo.org/api/records/15384890/files/NVCh_10h_continuous_trace.zip/content
print(file_to_download["key"] )
file_url = file_to_download["links"]["self"]
filename = file_to_download["key"] 
response = requests.get(file_url, stream=True)

with open(filename, "wb") as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)
print(f"Downloaded {filename}")

print(f"Extracting files...")

with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall(os.getcwd())  # or use a specific path
print(f"Files extracted")

NVCh_10h_continuous_trace.zip
Downloaded NVCh_10h_continuous_trace.zip
Extracting files...
Files extracted


After downloading we load the full seismic trace and the reference dataframe

In [19]:
filename = "NVCh_10h_continuous_trace.npy"

In [20]:
continuous_trace = np.load(filename)
reference = pd.read_csv("NVCh_10h_continuous_trace_reference.csv")
reference.value_counts("event_type")

event_type
LP    49
IC    37
TR    28
VT    28
AV    20
Name: count, dtype: int64

## Sliding window:

We apply our model in a sliding window setup. Stride can be manually selected and is by default set to the less computationally demanding value of 7000 (70 seconds). Batch size can also be selected and is by default 16

In [21]:
batch_size = 16
window_size = 8192
stride = 7000

activation_buffer = np.zeros([6,continuous_trace.shape[1]])
n_windows = ((continuous_trace.shape[1] - window_size) // stride) + 1
print_interval = int(n_windows/5)

In [22]:
%%time
with model_utils.SuppressPrint():
    model.eval()
window_index = 0
for batch in data_utils.generate_overlapping_batches(
    continuous_trace, window_size=window_size, stride=stride, batch_size=batch_size
):
    X = data_utils.patch_stacking_X(tensor(batch[:, 1:9, :])).to(device)
    output = model(X)
    output = data_utils.activation_unstacking(output, window_size, 256, 6)
    for idx in range(len(output)):
        offset = window_index * stride            
        probabilities = output[idx].detach().cpu().numpy()
        activation_buffer[:,offset:offset+window_size]=activation_buffer[:,offset:offset+window_size]+probabilities
        window_index += 1
        if window_index % print_interval == 0:
            print("windows processed:",window_index, "/", n_windows)


windows processed: 104 / 523
windows processed: 208 / 523
windows processed: 312 / 523
windows processed: 416 / 523
windows processed: 520 / 523
CPU times: total: 11.7 s
Wall time: 2.57 s


To obtain the time segmentation we must unstack and binarize the activations for each class:

In [23]:
max_indices = np.argmax(activation_buffer, axis=0)
binarized_activations = np.eye(len(activation_buffer))[max_indices].T

After unstacking and binarizing, we can find start and end times for the detected events. We can obtain a proxy for classification confidence by summing the non-binarized activations. In the following script we record model confidence for the first and second most probable class assignments:

In [24]:
indices_start_, indices_end_ = data_utils.get_events(binarized_activations[0])

label_map = {1: "VT", 2: "LP", 3: "TR", 4: "AV", 5: "IC"}
events = []

for idx_start_,idx_end_ in zip(indices_start_,indices_end_):
    class_act = activation_buffer[1:,idx_start_:idx_end_].sum(axis=1)
    if idx_end_-idx_start_>50:
        class_act = (class_act/class_act.sum())*100
        idx_desc = np.argsort(-class_act)+1      # or: arr.argsort()[::-1]
        pred_label = label_map[idx_desc[0]]
        second_pred_label = label_map[idx_desc[1]]
        conf_1 = class_act[idx_desc[0]-1]
        conf_2 = class_act[idx_desc[1]-1]
        events.append([pred_label, idx_start_, idx_end_,second_pred_label, np.round(conf_1,1),np.round(conf_2,1)])

cols = ["pred_label", "idx_start", "idx_end", "second_pred_label", "conf_1", "conf_2"]
predicted_df = pd.DataFrame(events, columns=cols)
predicted_df['duration']=predicted_df['idx_end']-predicted_df['idx_start']
print(len(predicted_df))
predicted_df = predicted_df.round(0)
predicted_df.head(len(predicted_df))

2283


,pred_label,idx_start,idx_end,second_pred_label,conf_1,conf_2,duration
0,AV,865,920,TR,100.0,0.0,55
1,AV,952,1024,TR,100.0,0.0,72
2,AV,1104,1280,TR,100.0,0.0,176
3,AV,1324,1536,TR,100.0,0.0,212
4,AV,1552,1792,LP,99.0,1.0,240
...,...,...,...,...,...,...,...
2278,AV,3647148,3647237,IC,51.0,49.0,89
2279,IC,3650684,3651096,TR,97.0,3.0,412
2280,IC,3651100,3651352,TR,95.0,5.0,252
2281,IC,3651473,3651548,VT,100.0,0.0,75


Detected events are then post-processed by merging same-class events separated by less than 2.5 seconds and discarding detections that do not fall into the limits estimated from the 24,493 labeled windows:

VT: 6–90 s, LP: 10–80 s, TR: $\ge$ 55 s, AV: 11–100 s, and IC: 3–16 s.

In [25]:
merged_detections = data_utils.merge_same_class_events_with_gap(
    predicted_df
)
merged_detections = merged_detections.round(0)
merged_detections.head(len(merged_detections))

,idx_start,idx_end,pred_label,second_pred_label,conf_1,conf_2
0,865,2560,AV,LP,100.0,0.0
1,8825,10656,AV,TR,100.0,0.0
2,26983,27130,AV,TR,89.0,5.0
3,30321,33195,AV,TR,100.0,0.0
4,36211,37048,AV,LP,77.0,23.0
...,...,...,...,...,...,...
631,3170616,3171000,VT,IC,100.0,0.0
632,3178772,3180295,VT,IC,69.0,19.0
633,3211938,3213449,VT,TR,77.0,12.0
634,3280623,3281888,VT,IC,66.0,17.0


In [26]:
bounds = {
    "VT": (600, 3000),
    "LP": (1000, 8000),
    "TR": (5500, 999990),
    "AV": (1100, 10000),
    "IC": (300, 1500),
}

merged_detections["duration"] = (
    merged_detections["idx_end"] - merged_detections["idx_start"]
)

bounds_df = (
    pd.DataFrame(bounds, index=["min_dur", "max_dur"])
    .T.reset_index()
    .rename(columns={"index": "pred_label"})
)
filtered_detections = (
    merged_detections.merge(bounds_df, on="pred_label", how="inner")
    .query("min_dur <= duration <= max_dur")
    .drop(columns=["min_dur", "max_dur"])
    .reset_index(drop=True)
)
filtered_detections = filtered_detections.round(1)
filtered_detections.head(len(filtered_detections))

,idx_start,idx_end,pred_label,second_pred_label,conf_1,conf_2,duration
0,865,2560,AV,LP,100.0,0.0,1695
1,8825,10656,AV,TR,100.0,0.0,1831
2,30321,33195,AV,TR,100.0,0.0,2874
3,66845,68493,AV,TR,100.0,0.0,1648
4,116969,118268,AV,IC,98.0,1.0,1299
...,...,...,...,...,...,...,...
325,3144767,3146237,VT,IC,100.0,0.0,1470
326,3178772,3180295,VT,IC,69.0,19.0,1523
327,3211938,3213449,VT,TR,77.0,12.0,1511
328,3280623,3281888,VT,IC,66.0,17.0,1265


Resulting predictions we can show through the *launch_event_viewer()* function. Note that reference is shown as a darker colored rectangle below the signal and detected events are shown as a lighter colored rectangle on top of it. Although detection is the same accross all stations we plot them at each one for ease of visualization.

In [ ]:
data_utils.launch_event_viewer(continuous_trace, reference, filtered_detections, activation_buffer, binarized_activations)

Output()